In [1]:
import base64
import csv
import json
import os
import requests
import tempfile
import urllib
import pprint
pp = pprint.PrettyPrinter()

import tensorflow as tf

import tfx
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2

import tensorflow_model_analysis as tfma

In [2]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.8.0
TFX version: 1.7.1


In [3]:
DATA_ROOT_DIR='data'
MODULE_FILE='trainer.py'
NOTEBOOK_FILEPATH='Pipeline_Development.ipynb'
PIPELINE_EXPORT_FILEPATH='pipeline_output/pipeline.py'
MODEL_EXPORT_FILEPATH='model_output'
PIPELINE_ROOT = os.path.join('pipeline_artifacts')
METADATA_PATH = os.path.join('metadata', 'metadata.db')

In [4]:
context = InteractiveContext(
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(METADATA_PATH)
)

In [5]:
example_gen = CsvExampleGen(input_base=DATA_ROOT_DIR)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 46
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [6]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

In [7]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 48
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [8]:
validator_gen = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
context.run(validator_gen)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 49
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [9]:
trainer_gen = Trainer(
    module_file=MODULE_FILE,
    examples=example_gen.outputs['examples'],
    train_args=trainer_pb2.TrainArgs(num_steps=200),
    eval_args=trainer_pb2.EvalArgs(num_steps=10)
)
context.run(trainer_gen)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying features.py -> build/lib
copying model.py -> build/lib
copying update_model.py -> build/lib
copying trainer.py -> build/lib
installing to /var/folders/yt/mm_9lw0j7nbb6ynlflgdw66h0000gn/T/tmp075e_vk6
running install
running install_lib
copying build/lib/features.py -> /var/folders/yt/mm_9lw0j7nbb6ynlflgdw66h0000gn/T/tmp075e_vk6
copying build/lib/model.py -> /var/folders/yt/mm_9lw0j7nbb6ynlflgdw66h0000gn/T/tmp075e_vk6
copying build/lib/update_model.py -> /var/folders/yt/mm_9lw0j7nbb6ynlflgdw66h0000gn/T/tmp075e_vk6
copying build/lib/trainer.py -> /var/folders/yt/mm_9lw0j7nbb6ynlflgdw66h0000gn/T/tmp075e_vk6
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manife

/Users/joe/opt/anaconda3/envs/tfx/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


Processing ./pipeline_artifacts/_wheels/tfx_user_code_Trainer-0.0+5fd88c011a4393c6bc11d035a9ea513c97eabe77ad224d5fb709363d337c8e26-py3-none-any.whl
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
200/200 [==============================] - 681s 3s/step - loss: 0.2870 - binary_accuracy: 0.8735 - val_loss: 0.1765 - val_binary_accuracy: 0.9200
Epoch 2/10
200/200 [==============================] - 853s 4s/step - loss: 0.2122 - binary_accuracy: 0.9208 - val_loss: 0.2238 - val_binary_accuracy: 0.9200
Epoch 3/10
 29/200 [===>..........................] - ETA: 11:40 - loss: 0.1564 - binary_accuracy: 0.9448

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/joe/opt/anaconda3/envs/tfx/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/yt/mm_9lw0j7nbb6ynlflgdw66h0000gn/T/ipykernel_19128/140860208.py", line 7, in <module>
    context.run(trainer_gen)
  File "/Users/joe/opt/anaconda3/envs/tfx/lib/python3.7/site-packages/tfx/orchestration/experimental/interactive/interactive_context.py", line 63, in run_if_ipython
    return fn(*args, **kwargs)
  File "/Users/joe/opt/anaconda3/envs/tfx/lib/python3.7/site-packages/tfx/orchestration/experimental/interactive/interactive_context.py", line 183, in run
    execution_id = launcher.launch().execution_id
  File "/Users/joe/opt/anaconda3/envs/tfx/lib/python3.7/site-packages/tfx/orchestration/launcher/base_component_launcher.py", line 203, in launch
    copy.deepcopy(execution_decision.exec_properties))
  File "/Users/joe/opt/anaconda3/envs/tfx/li

TypeError: object of type 'NoneType' has no len()

In [ ]:
from features import FEATURE_KEYS, LABEL_KEY


eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name='eval' and
        # remove the label_key. Note, if using a TFLite model, then you must set
        # model_type='tf_lite'.
        tfma.ModelSpec(label_key=LABEL_KEY)
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(
                    class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.5}),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': -1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
#         tfma.SlicingSpec(feature_keys=FEATURE_KEYS)
    ]
)


evaluator_gen = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer_gen.outputs['model'],
    eval_config=eval_config
)
context.run(evaluator_gen)

In [ ]:
pusher_gen = Pusher(
    model_blessing=evaluator_gen.outputs['blessing'],
    model=trainer_gen.outputs['model'],
    push_destination=tfx.proto.pusher_pb2.PushDestination(
        filesystem=tfx.proto.pusher_pb2.PushDestination.Filesystem(
            base_directory=MODEL_EXPORT_FILEPATH
        )
    )
)
context.run(pusher_gen)

# Methods Of Saving Model

In [10]:
# H5PY format
tf.keras.models.save_model(
    trainer_gen.outputs['model'],
    'model_output_h5/model.h5',
    overwrite=True,
    include_optimizer=True,
    save_format='h5',
    signatures=None,
    options=None,
    save_traces=True
)

AttributeError: 'OutputChannel' object has no attribute 'outputs'

In [ ]:
context.export_to_pipeline(notebook_filepath=NOTEBOOK_FILEPATH,
                           export_filepath=PIPELINE_EXPORT_FILEPATH,
                           runner_type='beam')

In [23]:
from tfx import types
from tfx.components.base import base_executor
from tfx.types.component_spec import ExecutionParameter
from tfx.types.component_spec import ChannelParameter

class S3ModelPusherComponentSpec(types.ComponentSpec):
  """ComponentSpec for Custom TFX Hello World Component."""

  PARAMETERS = {
      # These are parameters that will be passed in the call to
      # create an instance of this component.
#       'name': ExecutionParameter(type=Text),
#       's3_client': ExecutionParameter(type=)
  }
  INPUTS = {
      # This will be a dictionary with input artifacts, including URIs
      'model': ChannelParameter(type=types.standard_artifacts.Model),
  }
  OUTPUTS = {
      # This will be a dictionary which this component will populate
      'push_completed': ChannelParameter(type=types.standard_artifacts.Boolean),
  }


In [24]:
class S3ModelPusherExecutor(base_executor.BaseExecutor):
  """Executor for HelloComponent."""

  def Do(self, input_dict, output_dict, exec_properties):
    input_examples_uri = artifact_utils.get_split_uri(input_dict['model'], 'train')
    
    split_to_instance = {}
    for artifact in input_dict['input_data']:
      for split in json.loads(artifact.split_names):
        uri = artifact_utils.get_split_uri([artifact], split)
        split_to_instance[split] = uri

    for split, instance in split_to_instance.items():
      input_dir = instance
      output_dir = artifact_utils.get_split_uri(
          output_dict['output_data'], split)
      for filename in tf.io.gfile.listdir(input_dir):
        input_uri = os.path.join(input_dir, filename)
        output_uri = os.path.join(output_dir, filename)
        io_utils.copy_file(src=input_uri, dst=output_uri, overwrite=True)


NameError: name 'Dict' is not defined

In [ ]:

class S3ModelPusherComponent(base_component.BaseComponent):
  """Custom TFX Hello World Component."""

  SPEC_CLASS = S3ModelPusherComponentSpec
  EXECUTOR_SPEC = executor_spec.ExecutorClassSpec(S3ModelPusherExecutor.Executor)

  def __init__(self,
               input_data: types.Channel = None,
               output_data: types.Channel = None,
               name: Optional[Text] = None):
    if not output_data:
      examples_artifact = standard_artifacts.Examples()
      examples_artifact.split_names = input_data.get()[0].split_names
      output_data = channel_utils.as_channel([examples_artifact])

    spec = S3ModelPusherComponentSpec(input_data=input_data, output_data=output_data, name=name)
    
    super(S3ModelPusherComponent, self).__init__(spec=spec)
